In [25]:
import http.client
import json
import datetime
import pandas as pd
from statistics import mean

In [3]:
def getMonday(dateNow):
    today = dateNow
    today = today + datetime.timedelta(days=-today.weekday(), weeks=-1)
    return today.strftime("%d-%m-%Y")

In [8]:
def getOpenCovidPeruData(date):
    mondayDate = getMonday(date)
    conn = http.client.HTTPSConnection("open-covid-api-vwgk4ckqbq-uk.a.run.app")
    payload = ''
    headers = {}
    conn.request("GET", f"/api/semaforo?fecha={mondayDate}", payload, headers)
    res = conn.getresponse()
    data = res.read()
    jsonData = json.loads(data.decode('utf-8'))
    filteredData =  list(map((lambda value: {'region':value['region'],'appliedTests':value['avgTest'],'testPositivityPercentage':value['positividad'],'population':value['poblacion'],'bedsOccupiedPercentage':value['camasCovid'],'bedsUCIOccupiedPercentage':value['uci'],'confirmedCasesWeekly':round((value['poblacion']*value['incid_100'])/100000,2),'deathsCasesWeekly':round((value['poblacion']*value['fall_100'])/100000,2)}), jsonData[0]['regions']))
    df = pd.DataFrame(filteredData)
    return df

getOpenCovidPeruData(datetime.date.today()-datetime.timedelta(days=8))

,region,appliedTests,testPositivityPercentage,population,bedsOccupiedPercentage,bedsUCIOccupiedPercentage,confirmedCasesWeekly,deathsCasesWeekly
0,AMAZONAS,213.53,6.39,472993.0,9.09,26.32,63.00,9.98
1,ANCASH,514.42,2.73,1246249.0,16.51,68.33,173.98,5.98
2,APURIMAC,272.86,3.42,460316.0,6.81,20.51,44.01,5.98
3,AREQUIPA,1463.65,1.19,1556246.0,42.51,92.22,267.99,13.07
4,AYACUCHO,306.79,8.78,690053.0,10.53,55.00,177.00,6.00
5,CAJAMARCA,420.39,2.52,1603521.0,3.46,17.46,169.01,7.06
6,CALLAO,500.46,3.76,1147951.0,64.27,75.63,216.96,7.00
7,CUSCO,554.91,3.32,1460963.0,20.34,78.43,269.99,11.98
8,HUANCAVELICA,320.35,1.28,435152.0,5.88,0.00,19.02,8.01
9,HUANUCO,308.84,4.22,865820.0,23.66,65.22,112.99,5.97


In [27]:
def getWaqiData():
    humidityTotal = []
    pressureTotal = []
    pm10Total = []
    pm25Total = []
    temperatureTotal = []
    for department in ['@7580','@7577','@7578','@8780','@379']:
        conn = http.client.HTTPSConnection("api.waqi.info")
        payload = ''
        headers = {}
        conn.request("GET", f"/feed/{department}/?token=d91a4fcc7a848a548eaaeba48ae3a30f08e5d402", payload, headers)
        res = conn.getresponse()
        data = res.read()
        jsonData = json.loads(data.decode('utf-8'))
        jsonDataFiltered = jsonData['data']['iaqi']
        
        if 'h' in jsonDataFiltered.keys():
            humidityTotal.append(jsonDataFiltered['h']['v'])
        if 'p' in jsonDataFiltered.keys():
            pressureTotal.append(jsonDataFiltered['p']['v'])
        if 'pm10' in jsonDataFiltered.keys():
            pm10Total.append(jsonDataFiltered['pm10']['v'])
        if 'pm25' in jsonDataFiltered.keys():
            pm25Total.append(jsonDataFiltered['pm25']['v'])
        if 't' in jsonDataFiltered.keys():
            temperatureTotal.append(jsonDataFiltered['t']['v'])
    humidity = mean(humidityTotal)
    pressure = mean(pressureTotal)
    pm10 = mean(pm10Total)
    pm25 = mean(pm25Total)
    temperature = mean(temperatureTotal)
    return {'humidity':humidity,'pressure':pressure,'pm10':pm10,'pm2.5':pm25,'temperature':temperature}

getWaqiData()

{'humidity': 85.1,
 'pressure': 1004.02,
 'pm10': 48,
 'pm2.5': 96,
 'temperature': 17.54}